In [16]:
from util import get_num_lines, get_vocab, embed_sequence, get_word2idx_idx2word, get_embedding_matrix
from util import TextDatasetWithGloveElmoSuffix as TextDataset
from util import evaluate
from model import RNNSequenceClassifier

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.nn.functional as F

import csv
import h5py
import random
import math
import numpy as np
import matplotlib

matplotlib.use('Agg')  # to avoid the error: _tkinter.TclError: no display name and no $DISPLAY environment variable
matplotlib.use('tkagg')  # to display the graph on remote server
import matplotlib.pyplot as plt

In [17]:
print("PyTorch version:")
print(torch.__version__)
print("GPU Detected:")
print(torch.cuda.is_available())
using_GPU = True

PyTorch version:
1.2.0
GPU Detected:
True


In [18]:
Gutenberg_example = []

# normal version
with open('../Poetry/Gutenberg_example_test.csv') as f:
    lines = csv.reader(f)
    next(lines)
    for line in lines:
        Gutenberg_example.append([line[1].strip(), int(line[2]), int(line[3])])
        
print('Gutenberg_example  size: ', len(Gutenberg_example))
print(Gutenberg_example)

Gutenberg_example  size:  20
[['When on earth they fade and perish,', 4, 0], ['A larger rock then heaving from the plain,', 4, 0], ["Be born to Jacob's sacred blood. God kept", 8, 0], ['In bed at morrow, sleiping as I lay,', 5, 0], ["To hear her whisper woman's lore so well;", 1, 0], ['He had mittens, Minjekahwun,', 1, 0], ['He had often played his pipes among my hills', 3, 0], ['When night is one', 2, 0], ['My minde finds out a shade', 2, 0], ['As some one had sat unawares', 5, 0], ['On his own altar offering up my heart,', 4, 0], ['Softening the beauty that it could not hide.', 0, 0], ['And his step seemed light and gay;', 3, 0], ['I have been one who, in the glow of youth,', 2, 0], ["What for the maiden's hand shall be your speech,", 6, 0], ['The Dawn the rosy East has kissed;', 6, 0], ["Swim like a fish, and caper like a goat.'", 0, 0], ['I still might press thy silent heart,', 3, 0], ['From the blue east, growing', 5, 0], ["The shadow of her husband's head was dancing on the wall;

In [19]:
# vocab is a set of words
vocab = get_vocab(Gutenberg_example)
# two dictionaries. <PAD>: 0, <UNK>: 1
word2idx, idx2word = get_word2idx_idx2word(vocab)
# glove_embeddings a nn.Embeddings
glove_embeddings = get_embedding_matrix(word2idx, idx2word, normalization=False)
# elmo_embeddings
elmos_poetry = None
# suffix_embeddings: number of suffix tag is 2, and the suffix embedding dimension is 50
suffix_embeddings = nn.Embedding(2, 50)
print(suffix_embeddings)
print(suffix_embeddings(Variable(torch.LongTensor([0,1,0]))))

vocab size:  122


100%|██████████| 2196017/2196017 [00:24<00:00, 90402.54it/s]

Number of pre-trained word vectors loaded:  99
Embeddings mean:  -0.0034917686134576797
Embeddings stdev:  0.3376159965991974
Embedding(2, 50)
tensor([[-5.7709e-01,  6.1818e-04, -4.3676e-01, -1.4916e+00,  8.6609e-01,
          3.1666e-01, -9.6286e-01,  3.9210e-01,  1.1078e+00, -1.4485e+00,
         -2.1197e+00, -6.3186e-01,  2.0927e+00, -1.6487e+00, -5.3807e-01,
          8.3800e-01,  1.6360e+00, -1.6554e+00, -3.1370e-01, -3.9435e-01,
          1.9737e+00, -5.0938e-01, -3.4303e-02,  4.7136e-01,  3.8622e-01,
          2.2685e-03, -4.4109e-01, -1.4945e+00,  8.1011e-01,  7.3434e-02,
         -1.4654e+00, -8.7400e-01, -5.8039e-01, -2.2151e-01,  6.7763e-02,
         -7.8400e-01, -1.0579e+00,  8.0550e-01,  2.5418e-01,  7.6538e-01,
          3.6125e-01, -1.0264e+00, -6.5605e-02,  1.6008e+00, -3.5930e-01,
         -1.9328e+00,  1.1617e+00,  8.2853e-01,  2.4989e+00,  1.2158e+00],
        [-8.3020e-01,  6.3795e-01,  4.7662e-01,  1.7164e-01, -6.9459e-01,
         -3.3985e-01, -1.8781e+00, -2.4394

In [28]:
for example in Gutenberg_example:
    print(example[0] + str(example[1]) + str(example[2]))
    
    
embedded_Gutenberg_example = [[embed_sequence(example[0], example[1], word2idx, glove_embeddings,
                                   elmos_poetry, suffix_embeddings), example[2]]
                               for example in Gutenberg_example]
print(len(embedded_Gutenberg_example))
print(embedded_Gutenberg_example)


I still might press thy silent heart,30
The shadow of her husband's head was dancing on the wall;70
In bed at morrow, sleiping as I lay,50
Softening the beauty that it could not hide.00
He had mittens, Minjekahwun,10
From the blue east, growing50
Be born to Jacob's sacred blood. God kept80
My minde finds out a shade20
He had often played his pipes among my hills30
As some one had sat unawares50
A larger rock then heaving from the plain,40
Swim like a fish, and caper like a goat.'00
On his own altar offering up my heart,40
To hear her whisper woman's lore so well;10
When night is one20
The Dawn the rosy East has kissed;60
What for the maiden's hand shall be your speech,60
I have been one who, in the glow of youth,20
And his step seemed light and gay;30
When on earth they fade and perish,40


IndexError: list assignment index out of range

In [23]:
sentences = [example[0] for example in embedded_Gutenberg_example]
labels = [example[1] for example in embedded_Gutenberg_example]
print(sentences)
print('*' * 20)
print(labels)

NameError: name 'embedded_Gutenberg_example' is not defined

In [11]:
dataloader_sentences = TextDataset(sentences, labels)
print(dataloader_sentences)
poetry = DataLoader(dataset=dataloader_sentences, batch_size = 10, shuffle=True, collate_fn=TextDataset.collate_fn)
print(poetry)

NameError: name 'sentences' is not defined

In [8]:
net = torch.load('../models/LSTMSuffixElmoAtt_Poetry_fold_0_epoch_23.pt')
net.eval()

RNNSequenceClassifier(
  (rnn): LSTM(350, 300, batch_first=True, bidirectional=True)
  (attention_weights): Linear(in_features=600, out_features=1, bias=True)
  (output_projection): Linear(in_features=600, out_features=2, bias=True)
  (dropout_on_input_to_LSTM): Dropout(p=0.2, inplace=False)
  (dropout_on_input_to_linear_layer): Dropout(p=0, inplace=False)
)

In [26]:
output = []
for (text, lengths, labels) in poetry:
    text = Variable(text)
    lengths = Variable(lengths)
    if using_GPU:
        text = text.cuda()
        lengths = lengths.cuda()
#     print(text)
#     print('-'*10)
#     print(lengths)
    predicted = net(text, lengths)
    predicted_numpy = predicted.cpu().detach().numpy()
    output.append(predicted_numpy)
#     output.apeend(predicted.numpy())
#     print('*' * 10)
    print(predicted)
# print(output)

tensor([[-2.8059e+00, -6.2360e-02],
        [-5.0253e+00, -6.5916e-03],
        [-7.5224e+00, -5.4097e-04],
        [-6.5127e+00, -1.4856e-03],
        [-7.1204e+00, -8.0872e-04],
        [-5.1542e+00, -5.7917e-03],
        [-4.2729e+00, -1.4040e-02],
        [-1.4821e-02, -4.2191e+00],
        [-6.8655e+00, -1.0438e-03],
        [-6.5310e+00, -1.4586e-03]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)
tensor([[-7.0066e+00, -9.0623e-04],
        [-2.6466e-01, -1.4587e+00],
        [-6.5606e+00, -1.4160e-03],
        [-5.7835e+00, -3.0828e-03],
        [-2.2253e-01, -1.6119e+00],
        [-8.7452e-04, -7.0421e+00],
        [-6.9892e-02, -2.6956e+00],
        [-8.5779e+00, -1.8835e-04],
        [-9.4630e+00, -7.7724e-05],
        [-5.0754e-02, -3.0060e+00]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)
tensor([[-2.3847e-03, -6.0399e+00],
        [-5.2352e-02, -2.9758e+00],
        [-6.5541e-04, -7.3306e+00],
        [-6.3747e-02, -2.7845e+00],
        [-4.8779e+00, -7.6

In [24]:
# output = predicted.cpu().detach().numpy()
# print(output)

[[-1.0945559e-02 -4.5202951e+00]
 [-1.3060570e-02 -4.3446727e+00]
 [-7.4631367e+00 -5.7411194e-04]
 [-4.1817265e+00 -1.5389919e-02]
 [-2.6396787e-01 -1.4610100e+00]]


In [27]:
print(output)

[array([[-2.8058515e+00, -6.2359810e-02],
       [-5.0252666e+00, -6.5915585e-03],
       [-7.5224218e+00, -5.4097176e-04],
       [-6.5127125e+00, -1.4855862e-03],
       [-7.1204081e+00, -8.0871582e-04],
       [-5.1542349e+00, -5.7916641e-03],
       [-4.2728858e+00, -1.4039516e-02],
       [-1.4821291e-02, -4.2190962e+00],
       [-6.8655314e+00, -1.0437965e-03],
       [-6.5309882e+00, -1.4586449e-03]], dtype=float32), array([[-7.0066214e+00, -9.0622902e-04],
       [-2.6466477e-01, -1.4587065e+00],
       [-6.5606112e+00, -1.4159679e-03],
       [-5.7834725e+00, -3.0827522e-03],
       [-2.2252911e-01, -1.6118996e+00],
       [-8.7451935e-04, -7.0421438e+00],
       [-6.9891691e-02, -2.6955514e+00],
       [-8.5778646e+00, -1.8835068e-04],
       [-9.4629555e+00, -7.7724457e-05],
       [-5.0754309e-02, -3.0060282e+00]], dtype=float32), array([[-2.3846626e-03, -6.0398788e+00],
       [-5.2351713e-02, -2.9758310e+00],
       [-6.5541267e-04, -7.3305721e+00],
       [-6.3747287e-02

In [39]:
output_list = []
for i in range(len(output)):
    output_list.append(output[i])
print(output_list)

[array([[-2.8058515e+00, -6.2359810e-02],
       [-5.0252666e+00, -6.5915585e-03],
       [-7.5224218e+00, -5.4097176e-04],
       [-6.5127125e+00, -1.4855862e-03],
       [-7.1204081e+00, -8.0871582e-04],
       [-5.1542349e+00, -5.7916641e-03],
       [-4.2728858e+00, -1.4039516e-02],
       [-1.4821291e-02, -4.2190962e+00],
       [-6.8655314e+00, -1.0437965e-03],
       [-6.5309882e+00, -1.4586449e-03]], dtype=float32), array([[-7.0066214e+00, -9.0622902e-04],
       [-2.6466477e-01, -1.4587065e+00],
       [-6.5606112e+00, -1.4159679e-03],
       [-5.7834725e+00, -3.0827522e-03],
       [-2.2252911e-01, -1.6118996e+00],
       [-8.7451935e-04, -7.0421438e+00],
       [-6.9891691e-02, -2.6955514e+00],
       [-8.5778646e+00, -1.8835068e-04],
       [-9.4629555e+00, -7.7724457e-05],
       [-5.0754309e-02, -3.0060282e+00]], dtype=float32), array([[-2.3846626e-03, -6.0398788e+00],
       [-5.2351713e-02, -2.9758310e+00],
       [-6.5541267e-04, -7.3305721e+00],
       [-6.3747287e-02

In [40]:
print(output_list[1][1])

[-0.26466477 -1.4587065 ]
